# A More Complicated Example: Calculating Climatological Averages

The previous example showed one of the most basic GeoCAT-comp functions. This example showcases how some functions in GeoCAT-comp have a variety of options that can be selected using keyword arguments. The particular function we'll be looking at is `climatology_average`.

The datafile we're using contains surface temperature data at one location generated by numerous weather models in a model ensemble. The temporal resolution is 6 hours from January 1st 1990 to January 1st 1996. We want to take the data from each model and summarize it. We can do this in a few different ways with `climatology_average`. We can find the daily, monthly, or seasonal climatological averages.

## Dependencies
- geocat-comp
- geocat-datafiles
- xarray
- nc-time-axis

## Imports

In [ ]:
from geocat.comp import climatology_average

import geocat.datafiles as gdf
import xarray as xr

## Retrieving sample data from GeoCAT-datafiles
[GeoCAT-datafiles](https://github.com/ncar/geocat-datafiles) is a software package that makes it simple to download datafiles from the pacakge's GitHub repository. The repository holds data files of different types such as NetCDF and GRIB. They are used for GeoCAT tutorials, demonstrations, and test cases. You won't need to use GeoCAT-datafiles when working with your own data.

In [ ]:
nc_file = gdf.get('netcdf_files/atm.20C.hourly6-1990-1995-TS.nc')
nc_file

## Parsing the data with xarray
GeoCAT-comp is designed to work with [`xarray`](https://xarray.dev/) and [`numpy`](https://numpy.org/) data structures. Here we will be using `xarray` to import the netCDF data.

In [ ]:
ds = xr.open_dataset(nc_file)
ds

We can see here that the datafile has surface temperature data from 40 model ensemble members. Let's simplify things and just look at one model for now. We use `xarray`'s `DataArray.isel` method to pick out the data for the first member_id at index 0.

In [ ]:
ds = ds.isel(member_id=0)  # select one model from the ensemble
temp = ds.TS  # surface temperature data
temp

Now that we have the temperature data for the first ensemble member, let's plot it. `xarray` has built in methods to quickly plot data. The generated plot isn't publication quality, but it's a quick and dirty way to see what your dataset contains.

In [ ]:
temp.plot()

This plot shows that the temperature doesn't change much throughout the year. This makes sense since the data is for the location 0.4712N 180E, which is in the Pacific Ocean just north of the equator. There is some variation though, and it'd be interesting to know what the average temperature is in each month. We can calculate this using `climatology_average`.

## Using climatology_average
We can see in the [documentation](https://geocat-comp.readthedocs.io/en/latest/user_api/generated/geocat.comp.climatologies.climatology_average.html) that `climatology_average` calculates long term hourly, daily, monthly, or seasonal averages across all years in a given dataset. This is a good function to quickly summarize our data, so let's use it. 

<iframe src="https://geocat-comp.readthedocs.io/en/latest/user_api/generated/geocat.comp.climatologies.climatology_average.html" title="geocat-comp climatology_average documentation" width=100% height=600></iframe>

### Related NCL Functions
Notice at the bottom of the documentation page there is a box with links to related NCL and GeoCAT functions. This indicates all of the NCL functions that `climatology_average` is replicating through it's keyword arguments. We want something similar to the `clmMon` family of functions to summarize our data using monthly climate averages. We can see that our chosen GeoCAT function replicates that NCL capability.

<img src="../images/geocat-comp/climatology_average_note.png" width="100%" alt="A note on the climatology_average documentation page about related NCL functions clmDayHourTLL, clmDauHourTLLL, clmDayTLL, clmDayTLLL, clmMonLLLT, clmMonLLT, clmMonTLL, clmMonTLLL, month_to_season">

Let's summarize our data using monthly climate averages. Using the keyword argument `freq`, we can specify the time span for the average. The accepted options are `hour`, `day`, `month`, and `season`.

In [ ]:
monthly_temp = climatology_average(temp, freq='month')
monthly_temp

Notice how much smaller the time dimension is. Before, the time dimension had a length of 8761 points, one for every 6 hour interval over 5 years. Now, the time dimension has a length of 12, one for every month of the year. Notice that the value of the time coordinate is for the middle of each month in 1993. Why 1993? It's approximatly the middle year in the dataset.

In [ ]:
monthly_temp.plot()

When we plot this, the data is more granular and has even less variation in temperature from month to month. But we can now see which months are coldest and warmest on average for this location.

## Adjusting Parameters
Monthly climate averages are great, but what else can `climatology_average` do? We can also calculate hourly, daily, and seasonal climate averages. For this dataset, we don't have a fine enough temporal resolution to compute an hourly average, but we can compute daily averages. The only change to the function call is passing in `day` instead of `month`.

In [ ]:
daily_temp = climatology_average(temp, freq='day')
daily_temp

In [ ]:
daily_temp.plot()